# TP Partie 1 : Génération de Dataset Synthétique pour le Fine-tuning de LLM

## 📚 Contexte

Dans ce notebook, nous allons explorer la génération de datasets d'instruction synthétiques, une technique clé pour créer des données d'entraînement pour le fine-tuning de modèles de langage. Cette approche permet de :

- **Créer des données d'entraînement** sans annotation manuelle coûteuse
- **Adapter un modèle** à des domaines spécifiques
- **Augmenter la diversité** des données existantes

## 🎯 Objectifs

1. **Comprendre** le processus de génération de données synthétiques
2. **Implémenter** une pipeline de génération basée sur des documents sources
3. **Créer** un dataset au format instruction-response utilisable pour le fine-tuning
4. **Analyser** la qualité et la diversité des données générées

## 🔧 Architecture

Notre approche s'inspire de la méthodologie AWS pour la génération de données synthétiques :

1. **Extraction de contexte** : Nous utilisons le corpus Common Corpus (split français)
2. **Génération de questions** : Un LLM génère des questions pertinentes basées sur le contexte
3. **Génération de réponses** : Le même ou un autre LLM génère des réponses aux questions
4. **Formatage** : Structuration au format instruction-suivie pour le fine-tuning

## ⚙️ Configuration

Nous utilisons :
- **vLLM** : Serveur d'inférence haute performance (déjà configuré sur votre machine)
- **Qwen3-4B** : Modèle de base pour la génération
- **Common Corpus** : Dataset source en français

## 1. Import des bibliothèques et configuration

In [ ]:
import json
import random
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, asdict
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from datasets import load_dataset
from openai import OpenAI
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration visuelle
sns.set_theme()
plt.rcParams['figure.figsize'] = (10, 6)

## 2. Configuration du client vLLM

**Note importante** : Assurez-vous que votre serveur vLLM est lancé (`make vllm` dans un terminal séparé)

In [ ]:
# Configuration du client OpenAI pour vLLM
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="ecn-llm-token-update-this-secret"  # Token défini dans start_vllm.sh
)

# Test de connexion
try:
    models = client.models.list()
    print("✅ Connexion au serveur vLLM réussie")
    print(f"Modèle disponible : {models.data[0].id if models.data else 'Aucun'}")
except Exception as e:
    print(f"❌ Erreur de connexion : {e}")
    print("Assurez-vous d'avoir lancé le serveur vLLM avec 'make vllm'")

## 3. Chargement du dataset source

Nous utilisons le **Common Corpus** en français comme base pour générer nos instructions synthétiques.

In [ ]:
# Chargement du dataset
print("Chargement du dataset Common Corpus (split français)...")
dataset = load_dataset(
    "PleIAs/common_corpus",
    split="train",
    streaming=True  # Streaming pour économiser la mémoire
)

# Échantillonnage pour le TP (ajustez selon vos besoins)
SAMPLE_SIZE = 100  # Nombre de documents à traiter
documents = []

print(f"Extraction de {SAMPLE_SIZE} documents...")
for i, doc in enumerate(tqdm(dataset, total=SAMPLE_SIZE)):
    if i >= SAMPLE_SIZE:
        break
    # Filtrer les documents trop courts ou trop longs
    if doc.get('text') and 100 < len(doc['text']) < 2000:
        documents.append(doc['text'])

print(f"\n📊 Documents collectés : {len(documents)}")
print(f"Longueur moyenne : {np.mean([len(d) for d in documents]):.0f} caractères")

## 4. Définition des structures de données

Nous utilisons des dataclasses pour structurer nos données de manière claire et typée.

In [ ]:
@dataclass
class InstructionPair:
    """Structure pour une paire instruction-réponse"""
    instruction: str
    response: str
    context: str
    metadata: Dict = None
    
    def to_dict(self) -> Dict:
        return asdict(self)
    
    def to_alpaca_format(self) -> Dict:
        """Conversion au format Alpaca standard"""
        return {
            "instruction": self.instruction,
            "input": "",  # Contexte optionnel
            "output": self.response
        }

## 5. Pipeline de génération synthétique

### 5.1 Prompts pour la génération

Nous définissons des prompts soigneusement conçus pour générer des questions et réponses de qualité.

In [ ]:
# Templates de prompts pour la génération
QUESTION_GENERATION_PROMPT = """Tu es un assistant pédagogique expert. À partir du texte suivant, génère {num_questions} questions pertinentes et variées en français.

Les questions doivent :
- Être claires et précises
- Couvrir différents aspects du texte
- Varier en complexité (compréhension, analyse, synthèse)
- Être formulées en français correct

Texte source :
{context}

Génère les questions au format JSON :
["question1", "question2", ...]
"""

ANSWER_GENERATION_PROMPT = """Tu es un assistant expert. Réponds à la question suivante de manière claire, précise et pédagogique en te basant sur le contexte fourni.

Contexte :
{context}

Question :
{question}

Réponse :
"""

### 5.2 Fonctions de génération

In [ ]:
def generate_questions(context: str, num_questions: int = 3) -> List[str]:
    """
    Génère des questions basées sur un contexte donné.
    
    Args:
        context: Le texte source
        num_questions: Nombre de questions à générer
    
    Returns:
        Liste de questions générées
    """
    try:
        response = client.chat.completions.create(
            model="Qwen/Qwen3-4B",
            messages=[
                {"role": "system", "content": "Tu es un assistant qui génère des questions pédagogiques."},
                {"role": "user", "content": QUESTION_GENERATION_PROMPT.format(
                    context=context[:1500],  # Limiter la longueur du contexte
                    num_questions=num_questions
                )}
            ],
            temperature=0.7,
            max_tokens=500
        )
        
        # Extraction des questions du JSON
        content = response.choices[0].message.content
        
        # Tentative de parsing JSON
        try:
            # Chercher le JSON dans la réponse
            import re
            json_match = re.search(r'\[.*\]', content, re.DOTALL)
            if json_match:
                questions = json.loads(json_match.group())
                return questions[:num_questions]
        except:
            pass
        
        # Fallback : extraction simple des questions
        lines = content.split('\n')
        questions = [line.strip('- ').strip() for line in lines 
                    if line.strip() and '?' in line]
        return questions[:num_questions] if questions else ["Quelle est l'idée principale de ce texte ?"]
        
    except Exception as e:
        print(f"Erreur lors de la génération de questions : {e}")
        return ["Quelle est l'idée principale de ce texte ?"]


def generate_answer(context: str, question: str) -> str:
    """
    Génère une réponse à une question basée sur un contexte.
    
    Args:
        context: Le texte source
        question: La question à répondre
    
    Returns:
        Réponse générée
    """
    try:
        response = client.chat.completions.create(
            model="Qwen/Qwen3-4B",
            messages=[
                {"role": "system", "content": "Tu es un assistant expert qui répond de manière claire et pédagogique."},
                {"role": "user", "content": ANSWER_GENERATION_PROMPT.format(
                    context=context[:1500],
                    question=question
                )}
            ],
            temperature=0.3,  # Température plus basse pour des réponses cohérentes
            max_tokens=300
        )
        
        return response.choices[0].message.content.strip()
        
    except Exception as e:
        print(f"Erreur lors de la génération de réponse : {e}")
        return "Je ne peux pas répondre à cette question basée sur le contexte fourni."

### 5.3 Pipeline complète de génération

In [ ]:
def generate_instruction_pairs(
    documents: List[str], 
    questions_per_doc: int = 2,
    max_docs: Optional[int] = None
) -> List[InstructionPair]:
    """
    Génère des paires instruction-réponse à partir de documents.
    
    Args:
        documents: Liste de documents sources
        questions_per_doc: Nombre de questions par document
        max_docs: Nombre maximum de documents à traiter
    
    Returns:
        Liste de paires instruction-réponse
    """
    instruction_pairs = []
    docs_to_process = documents[:max_docs] if max_docs else documents
    
    for doc in tqdm(docs_to_process, desc="Traitement des documents"):
        # Générer les questions
        questions = generate_questions(doc, questions_per_doc)
        
        # Générer les réponses pour chaque question
        for question in questions:
            if question and len(question) > 10:  # Filtre de qualité basique
                answer = generate_answer(doc, question)
                
                if answer and len(answer) > 20:  # Filtre de qualité pour la réponse
                    pair = InstructionPair(
                        instruction=question,
                        response=answer,
                        context=doc[:500],  # Garder un extrait du contexte
                        metadata={
                            "source": "common_corpus_fr",
                            "generation_method": "vllm_qwen3_4b"
                        }
                    )
                    instruction_pairs.append(pair)
    
    return instruction_pairs

## 6. Génération du dataset synthétique

**⚠️ Note** : Cette étape peut prendre plusieurs minutes selon le nombre de documents et la vitesse de votre GPU.

In [ ]:
# Paramètres de génération
NUM_DOCS_TO_PROCESS = 10  # Commencez petit pour tester
QUESTIONS_PER_DOC = 2

print(f"🚀 Démarrage de la génération synthétique")
print(f"   - Documents à traiter : {NUM_DOCS_TO_PROCESS}")
print(f"   - Questions par document : {QUESTIONS_PER_DOC}")
print(f"   - Paires attendues : ~{NUM_DOCS_TO_PROCESS * QUESTIONS_PER_DOC}\n")

# Génération
synthetic_pairs = generate_instruction_pairs(
    documents,
    questions_per_doc=QUESTIONS_PER_DOC,
    max_docs=NUM_DOCS_TO_PROCESS
)

print(f"\n✅ Génération terminée : {len(synthetic_pairs)} paires créées")

## 7. Analyse et visualisation du dataset généré

In [ ]:
# Conversion en DataFrame pour l'analyse
df = pd.DataFrame([pair.to_dict() for pair in synthetic_pairs])

# Statistiques de base
print("📊 Statistiques du dataset généré :")
print(f"   - Nombre total de paires : {len(df)}")
print(f"   - Longueur moyenne des instructions : {df['instruction'].str.len().mean():.0f} caractères")
print(f"   - Longueur moyenne des réponses : {df['response'].str.len().mean():.0f} caractères")

# Affichage d'exemples
print("\n📝 Exemples de paires générées :\n")
for i in range(min(3, len(synthetic_pairs))):
    print(f"--- Exemple {i+1} ---")
    print(f"Instruction : {synthetic_pairs[i].instruction}")
    print(f"Réponse : {synthetic_pairs[i].response[:200]}...\n")

In [ ]:
# Visualisations
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution des longueurs d'instructions
axes[0].hist(df['instruction'].str.len(), bins=20, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Longueur (caractères)')
axes[0].set_ylabel('Fréquence')
axes[0].set_title('Distribution des longueurs d\'instructions')
axes[0].axvline(df['instruction'].str.len().mean(), color='red', 
                linestyle='--', label=f'Moyenne: {df["instruction"].str.len().mean():.0f}')
axes[0].legend()

# Distribution des longueurs de réponses
axes[1].hist(df['response'].str.len(), bins=20, edgecolor='black', alpha=0.7, color='green')
axes[1].set_xlabel('Longueur (caractères)')
axes[1].set_ylabel('Fréquence')
axes[1].set_title('Distribution des longueurs de réponses')
axes[1].axvline(df['response'].str.len().mean(), color='red', 
                linestyle='--', label=f'Moyenne: {df["response"].str.len().mean():.0f}')
axes[1].legend()

plt.tight_layout()
plt.show()

## 8. Sauvegarde du dataset

Nous sauvegardons le dataset dans plusieurs formats pour faciliter son utilisation ultérieure.

In [ ]:
import os
from pathlib import Path

# Création du dossier de sortie
output_dir = Path("../data/synthetic")
output_dir.mkdir(parents=True, exist_ok=True)

# Format Alpaca (standard pour le fine-tuning)
alpaca_data = [pair.to_alpaca_format() for pair in synthetic_pairs]
alpaca_path = output_dir / "synthetic_dataset_alpaca.json"
with open(alpaca_path, 'w', encoding='utf-8') as f:
    json.dump(alpaca_data, f, ensure_ascii=False, indent=2)
print(f"✅ Dataset au format Alpaca sauvegardé : {alpaca_path}")

# Format CSV pour l'analyse
csv_path = output_dir / "synthetic_dataset.csv"
df.to_csv(csv_path, index=False, encoding='utf-8')
print(f"✅ Dataset CSV sauvegardé : {csv_path}")

# Format JSONL (ligne par ligne, efficace pour le streaming)
jsonl_path = output_dir / "synthetic_dataset.jsonl"
with open(jsonl_path, 'w', encoding='utf-8') as f:
    for pair in synthetic_pairs:
        f.write(json.dumps(pair.to_dict(), ensure_ascii=False) + '\n')
print(f"✅ Dataset JSONL sauvegardé : {jsonl_path}")

## 9. Contrôle qualité et filtrage

Il est important de vérifier la qualité des données générées avant de les utiliser pour le fine-tuning.

In [ ]:
def quality_check(pair: InstructionPair) -> Tuple[bool, List[str]]:
    """
    Vérifie la qualité d'une paire instruction-réponse.
    
    Returns:
        (pass_check, list_of_issues)
    """
    issues = []
    
    # Vérifications de base
    if len(pair.instruction) < 10:
        issues.append("Instruction trop courte")
    if len(pair.response) < 20:
        issues.append("Réponse trop courte")
    if len(pair.instruction) > 500:
        issues.append("Instruction trop longue")
    if len(pair.response) > 1000:
        issues.append("Réponse trop longue")
    
    # Vérification de la présence de caractères spéciaux problématiques
    if any(char in pair.instruction + pair.response for char in ['\x00', '\r']):
        issues.append("Caractères spéciaux détectés")
    
    # Vérification de la cohérence
    if '?' not in pair.instruction:
        issues.append("L'instruction ne contient pas de question")
    
    return len(issues) == 0, issues

# Application du contrôle qualité
quality_results = []
for pair in synthetic_pairs:
    passed, issues = quality_check(pair)
    quality_results.append({
        'passed': passed,
        'issues': issues,
        'pair': pair
    })

# Statistiques de qualité
passed_count = sum(1 for r in quality_results if r['passed'])
print(f"📊 Résultats du contrôle qualité :")
print(f"   - Paires validées : {passed_count}/{len(quality_results)} ({100*passed_count/len(quality_results):.1f}%)")

# Affichage des problèmes les plus fréquents
all_issues = [issue for r in quality_results for issue in r['issues']]
if all_issues:
    from collections import Counter
    issue_counts = Counter(all_issues)
    print("\n🔍 Problèmes détectés :")
    for issue, count in issue_counts.most_common(5):
        print(f"   - {issue}: {count} occurrences")

In [ ]:
# Sauvegarde du dataset filtré
filtered_pairs = [r['pair'] for r in quality_results if r['passed']]

if filtered_pairs:
    filtered_path = output_dir / "synthetic_dataset_filtered.json"
    with open(filtered_path, 'w', encoding='utf-8') as f:
        json.dump(
            [pair.to_alpaca_format() for pair in filtered_pairs],
            f,
            ensure_ascii=False,
            indent=2
        )
    print(f"\n✅ Dataset filtré sauvegardé : {filtered_path}")
    print(f"   Contient {len(filtered_pairs)} paires de haute qualité")

## 10. Préparation pour le fine-tuning

### Recommandations pour l'étape suivante

Votre dataset synthétique est maintenant prêt pour le fine-tuning ! Voici quelques recommandations :

1. **Taille du dataset** : 
   - Minimum recommandé : 100-500 paires pour un fine-tuning basique
   - Idéal : 1000-5000 paires pour de meilleurs résultats

2. **Diversité** :
   - Variez les types de questions (factuelle, analytique, créative)
   - Utilisez différents domaines de contexte

3. **Qualité** :
   - Privilégiez la qualité à la quantité
   - Vérifiez manuellement un échantillon

4. **Format** :
   - Le format Alpaca est directement utilisable avec la plupart des frameworks
   - Gardez une trace du contexte original pour l'évaluation

In [ ]:
# Résumé final
print("🎉 Génération de dataset synthétique terminée !\n")
print("📊 Résumé :")
print(f"   - Documents sources traités : {NUM_DOCS_TO_PROCESS}")
print(f"   - Paires générées : {len(synthetic_pairs)}")
print(f"   - Paires de haute qualité : {len(filtered_pairs) if filtered_pairs else 0}")
print(f"\n📁 Fichiers créés :")
print(f"   - {alpaca_path.name} (format Alpaca pour fine-tuning)")
print(f"   - {csv_path.name} (format CSV pour analyse)")
print(f"   - {jsonl_path.name} (format JSONL pour streaming)")
if filtered_pairs:
    print(f"   - {filtered_path.name} (dataset filtré)")

print("\n🚀 Prochaine étape : Notebook 02 - Fine-tuning avec LoRA")

## 📚 Pour aller plus loin

### Améliorations possibles

1. **Diversification des prompts** :
   - Créer plusieurs templates de prompts
   - Adapter les prompts selon le type de contenu

2. **Techniques avancées** :
   - Self-instruct : Utiliser le modèle pour générer ses propres instructions
   - Constitutional AI : Ajouter des contraintes éthiques
   - Évolution : Faire évoluer les instructions progressivement

3. **Validation** :
   - Utiliser un modèle différent pour valider les réponses
   - Implémenter un système de scoring automatique
   - Cross-validation avec des annotateurs humains

4. **Optimisation** :
   - Batch processing pour accélérer la génération
   - Caching des résultats intermédiaires
   - Parallélisation des requêtes

### Exercices proposés

1. **Modifier les prompts** pour générer des instructions dans un domaine spécifique (médical, juridique, technique)
2. **Implémenter un système de scoring** pour évaluer automatiquement la qualité des paires
3. **Créer une pipeline de validation** avec un second modèle
4. **Expérimenter avec différentes températures** et analyser l'impact sur la diversité